In [3]:
import os
import glob
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.warp import transform_bounds
from rasterio.vrt import WarpedVRT
from shapely.geometry import box
from scipy.ndimage import gaussian_filter
from osgeo import gdal


def create_vector_bounding_boxes(raster_files, output_gpkg):
    """Generate bounding boxes for each raster and save as a GPKG."""
    features = []
    for raster in raster_files:
        with rasterio.open(raster) as src:
            bounds = transform_bounds(src.crs, "EPSG:4326", *src.bounds)
            features.append({"geometry": box(*bounds), "filename": os.path.basename(raster)})
    
    gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")
    gdf.to_file(output_gpkg, driver="GPKG")

def create_global_vrt(raster_files, output_vrt):
    """Create a VRT file from input rasters."""
    vrt_options = gdal.BuildVRTOptions(resampleAlg="bilinear", addAlpha=True)
    gdal.BuildVRT(output_vrt, raster_files, options=vrt_options)

def find_neighbors(raster_files):
    """Determine neighboring rasters for each tile."""
    neighbors = {}
    tile_map = {}
    
    for raster in raster_files:
        with rasterio.open(raster) as src:
            bounds = src.bounds
            tile_map[os.path.basename(raster)] = bounds
    
    for tile, bounds in tile_map.items():
        left, bottom, right, top = bounds
        
        neighbors[tile] = {
            "left": None, "right": None, "top": None, "bottom": None
        }
        
        for other_tile, other_bounds in tile_map.items():
            if tile == other_tile:
                continue
            
            ol, ob, or_, ot = other_bounds
            
            if np.isclose(left, or_):
                neighbors[tile]["left"] = other_tile
            if np.isclose(right, ol):
                neighbors[tile]["right"] = other_tile
            if np.isclose(top, ob):
                neighbors[tile]["top"] = other_tile
            if np.isclose(bottom, ot):
                neighbors[tile]["bottom"] = other_tile
    
    return neighbors

def smooth_edges(raster_files, neighbors, radius=100):
    """Smooth the edges of rasters where they have neighbors to ensure continuity."""
    smoothed_rasters = {}
    
    for raster in raster_files:
        tile_name = os.path.basename(raster)
        with rasterio.open(raster) as src:
            data = src.read(1)
            profile = src.profile
            smoothed_data = data.copy()
            
            for direction, neighbor_tile in neighbors[tile_name].items():
                if neighbor_tile is None:
                    continue
                
                with rasterio.open(neighbor_tile) as neighbor_src:
                    neighbor_data = neighbor_src.read(1)
                    
                    if direction == "left":
                        blend_zone = slice(0, radius)
                        smoothed_data[:, blend_zone] = gaussian_filter(
                            (data[:, blend_zone] + neighbor_data[:, -radius:]) / 2, sigma=5)
                    elif direction == "right":
                        blend_zone = slice(-radius, None)
                        smoothed_data[:, blend_zone] = gaussian_filter(
                            (data[:, blend_zone] + neighbor_data[:, :radius]) / 2, sigma=5)
                    elif direction == "top":
                        blend_zone = slice(0, radius)
                        smoothed_data[blend_zone, :] = gaussian_filter(
                            (data[blend_zone, :] + neighbor_data[-radius:, :]) / 2, sigma=5)
                    elif direction == "bottom":
                        blend_zone = slice(-radius, None)
                        smoothed_data[blend_zone, :] = gaussian_filter(
                            (data[blend_zone, :] + neighbor_data[:radius, :]) / 2, sigma=5)
            
            smoothed_rasters[tile_name] = smoothed_data
            
            with rasterio.open(f"smoothed_{tile_name}", "w", **profile) as dst:
                dst.write(smoothed_data, 1)
    
    return smoothed_rasters


# make modifucation to the code so it works without any error, check for potential error and fix them
# show me the process you are using
def hharm_workflow(outdir,raster_files):
    """Main execution function."""

    
    vector_output = os.path.join(outdir, "raster_bounding_boxes.gpkg")
    vrt_output = os.path.join(outdir, "global_mosaic.vrt")
    
    create_vector_bounding_boxes(raster_files, vector_output)
    create_global_vrt(raster_files, vrt_output)
    neighbors = find_neighbors(raster_files)
    smoothed_rasters = smooth_edges(raster_files, neighbors)
    
    print("Processing complete.")

simple solution for now 
- create vrt and clip from it , and use gpkg to check the boundary

In [1]:
DTILE_PATH = ""

In [ ]:
roi = 'RNG'
roi_dipath = f"{DTILE_PATH}/tiles/{roi}"
hharm_dpath = f"{DTILE_PATH}/hharm/{roi}"
os.makedirs(hharm_dpath, exist_ok=True)
files = glob.glob(f'{roi_dipath}/*.tif'); print(len(files))
files